# WhisperX Cloud - Minimal Install

Quick setup: Mamba -> pip install whisperx -> run server

Steps:
1. Set GPU: Runtime -> Change runtime type -> GPU
2. Paste ngrok token in Step 5
3. Run All (Step 5 will block and keep running)
4. Press Ctrl+C in Step 5 output to stop

In [ ]:
# Step 1: Install Mamba and create environment
import os, sys, subprocess

ENV_PATH = '/content/conda-envs/whisperx' if 'google.colab' in sys.modules else '/kaggle/working/conda-envs/whisperx' if os.path.exists('/kaggle') else os.path.expanduser('~/conda-envs/whisperx')

print("Installing Mamba...")
if not os.path.exists(os.path.expanduser('~/miniforge3/bin/mamba')):
    !wget -q -O /tmp/miniforge.sh https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
    !bash /tmp/miniforge.sh -b -p ~/miniforge3

MAMBA = os.path.expanduser('~/miniforge3/bin/mamba')

print(f"Creating env: {ENV_PATH}")
if os.path.exists(ENV_PATH):
    !{MAMBA} remove -p {ENV_PATH} --all -y -q 2>/dev/null || true

!{MAMBA} create -p {ENV_PATH} -c conda-forge -y python=3.10 ffmpeg git pip
print("Env created!")

In [ ]:
# Step 2: pip install whisperx
PYTHON = f"{ENV_PATH}/bin/python"

print("Installing whisperx...")
print("This may take a few minutes...")

# Install whisperx and required packages
!{PYTHON} -m pip install whisperx pyngrok librosa fastapi uvicorn python-multipart

# Install ngrok binary (required by pyngrok)
!{PYTHON} -m pyngrok install

print("\nDone!")

In [ ]:
# Step 3: Verify
print("Verifying...")
for pkg in ['torch', 'whisperx', 'fastapi', 'librosa']:
    r = subprocess.run([PYTHON, '-c', f'import {pkg}; print("OK")'], capture_output=True)
    print(f"  {'OK' if r.returncode == 0 else 'FAIL'} {pkg}")

import json
with open('.conda_python_path', 'w') as f:
    json.dump({'python_path': PYTHON, 'env_prefix': ENV_PATH}, f)

In [ ]:
# Step 4: Download server
import urllib.request
url = "https://raw.githubusercontent.com/infinite-gaming-studio/VideoLingo/main/whisperx_cloud/whisperx_server.py"
if not os.path.exists('whisperx_server.py'):
    urllib.request.urlretrieve(url, 'whisperx_server.py')
    print("Downloaded")
else:
    print("Exists")

In [ ]:
# Step 5: Start server with ngrok (BLOCKING - keeps running)
NGROK_TOKEN = ""  # <-- PASTE YOUR NGROK TOKEN HERE

if not NGROK_TOKEN:
    raise ValueError("Please paste your ngrok token above! Get it from https://dashboard.ngrok.com/get-started/your-authtoken")

import subprocess, threading, sys, time
sys.path.insert(0, f"{ENV_PATH}/lib/python3.10/site-packages")
from pyngrok import ngrok, conf

# Set token and connect
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(8000, "http")
print(f"🌐 Public URL: {public_url}")
print(f"\n📋 Copy to VideoLingo config:")
print(f"   whisperX_cloud_url: '{public_url}'")
print("\n🚀 Starting server... (Press Ctrl+C to stop)\n")

# Start server with real-time log streaming
proc = subprocess.Popen(
    [PYTHON, 'whisperx_server.py', '--port', '8000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1
)

def stream_output(pipe, prefix):
    """Stream subprocess output with prefix"""
    for line in iter(pipe.readline, ''):
        if line:
            print(f"[{prefix}] {line}", end='')
    pipe.close()

# Start log streaming threads
threading.Thread(target=stream_output, args=(proc.stdout, "SERVER"), daemon=True).start()
threading.Thread(target=stream_output, args=(proc.stderr, "ERROR"), daemon=True).start()

# Wait for server to start
time.sleep(3)
print("\n✅ Server is running!\n")

# Keep running until interrupted
try:
    while proc.poll() is None:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n\n🛑 Shutting down...")
    proc.terminate()
    proc.wait(timeout=5)
    ngrok.disconnect(public_url)
    print("✅ Stopped")